In [11]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# X_train = torch.rand(700, 3, 3)
# Y_train = (torch.sum(X_train>0.5, dim=(1,2))>4).float()
# X_test = torch.rand(300, 3, 3)
# Y_test = (torch.sum(X_test>0.5, dim=(1,2))>4).float()

# data = {}
# data['x_train']=X_train.numpy().tolist()
# data['y_train']=Y_train.numpy().tolist()
# data['x_test']=X_test.numpy().tolist()
# data['y_test']=Y_test.numpy().tolist()

# with open('data.json', 'w') as fp:
#     json.dump(data, fp)

with open('data.json') as json_file:
    data = json.load(json_file)

X_train = torch.FloatTensor(data['x_train'])
Y_train = torch.FloatTensor(data['y_train'])
X_test = torch.FloatTensor(data['x_test'])
Y_test = torch.FloatTensor(data['y_test'])



In [12]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 3, 3, padding=1)
        self.mp1 = nn.MaxPool2d(3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(3, 3, 3, padding=1)
        self.mp2 = nn.MaxPool2d(3, stride=1, padding=1)
        self.fc = nn.Linear(27, 1)

    def forward(self, x):
        in_size = x.size(0)
        x = self.mp1(F.relu(self.conv1(x)))
        x = self.mp2(F.relu(self.conv2(x)))
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

model = ConvNet()

criterion = nn.MSELoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
# 
# Training
# 
results = {} #{iter: [loss, accuracy]}
for epoch in range(1000):  # loop over the dataset multiple times

    running_loss = 0.0
    for i in range(len(Y_train)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = X_train[i], torch.tensor([Y_train[i]])
        inputs = inputs.view(1,1,len(inputs), len(inputs))
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
#         print(outputs.squeeze(0), labels)
        loss = criterion(outputs.squeeze(0), labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    
    if epoch % 10 == 9:
        print ("Epoch: %d, Loss: %.3f"%(epoch+1, running_loss))
    
        correct = 0.
        total = float(len(Y_test))

        with torch.no_grad():
            for i in range(len(Y_test)):
                images, labels = X_test[i], torch.tensor(Y_test[i])
                images = images.view(1,1,len(images), len(images))

                output = model(images)

                predicted = output.data > 0.5

                correct += (int(predicted[0]) == int(labels))
                
                accuracy = 100. * correct / total
        print('Accuracy of the network: %.2f %%' % (accuracy))
        
        results[epoch+1] = [running_loss, accuracy]

Epoch: 10, Loss: 14.519
Accuracy of the network: 60.00 %


/miniconda3/envs/camo/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch: 20, Loss: 10.263
Accuracy of the network: 76.67 %
Epoch: 30, Loss: 6.658
Accuracy of the network: 90.00 %
Epoch: 40, Loss: 5.503
Accuracy of the network: 90.00 %
Epoch: 50, Loss: 5.017
Accuracy of the network: 90.00 %
Epoch: 60, Loss: 4.748
Accuracy of the network: 90.00 %
Epoch: 70, Loss: 4.510
Accuracy of the network: 90.00 %
Epoch: 80, Loss: 4.227
Accuracy of the network: 90.00 %
Epoch: 90, Loss: 3.919
Accuracy of the network: 93.33 %
Epoch: 100, Loss: 3.677
Accuracy of the network: 93.33 %
Epoch: 110, Loss: 3.500
Accuracy of the network: 93.33 %
Epoch: 120, Loss: 3.213
Accuracy of the network: 96.67 %
Epoch: 130, Loss: 2.960
Accuracy of the network: 96.67 %
Epoch: 140, Loss: 2.700
Accuracy of the network: 96.67 %
Epoch: 150, Loss: 2.383
Accuracy of the network: 96.67 %
Epoch: 160, Loss: 2.187
Accuracy of the network: 96.67 %
Epoch: 170, Loss: 1.929
Accuracy of the network: 96.67 %
Epoch: 180, Loss: 1.886
Accuracy of the network: 96.67 %
Epoch: 190, Loss: 1.610
Accuracy of th

In [14]:
with open('results.json', 'w') as fp:
    json.dump(results, fp)
    
    
    
# correct = 0.
# total = float(len(Y_test))

# outputs = []
# with torch.no_grad():
#     for i in range(len(Y_test)):
#         images, labels = X_test[i], torch.tensor(Y_test[i])
#         images = images.view(1,1,len(images), len(images))
        
#         output = model(images)
#         outputs.append(output)

#         predicted = output.data > 0.5

#         correct += (int(predicted[0]) == int(labels))

# print('Accuracy of the network on the %d test images: %d %%' % (len(Y_test),
#     100 * correct / total))